# 4장 : 머신러닝의 기본요소
---

### 구성
1. 머신러닝의 네가지 분류
2. 머신 러닝 모델 평가
3. 데이터 전처리, 특성공학, 특성 학습
4. 과대 적합과 과소 적합
5. 보편적인 머신러닝 작업 흐름
6. 요약


### 이 장에서 다룰 핵심 내용
* 분류와 회귀 이외의 머신 러닝 형태
* 머신 러닝 모델의 올바른 평가 과정
* 딥러닝을 위한 데이터 전처리
* 특성 공학
* 과대적합 문제 해결
* 머신 러닝 문제를 다루는 일반적인 작업 흐름

## 4.2 머신러닝 모델 평가
훈련에서 사용된 동일ㅇ한 데이터로 모델을 평가하지 않은 이유는 **몇번의 에포크 이후로 과대적합되기 시작했기 때문**입니다. 머신러닝의 목표는 처음 본 데이터에서 잘 작동하는 일반화된 모델을 얻는 것 입니다. 여기에서 과대적합은 주요 장애물입니다. 관측할 수 있는 것만 제어할 수 있으므로 모델의 일반화 성능에 대한 신뢰할 수 있는 측정방법이 아주 중요합니다.

### 훈련, 검증, 테스트 세트
모델 평가의 핵심은 *가용한 데이터를 항상 훈련, 검증 테스트 3개의 세트로 나누는 것*입니다. 훈련 세트에서 모델을 훈련하고 검증 세트에서 모델을 평가합니다. 모델을 출시할 준비가 되면 테스트세트에서 최종적으로 딱 한 번 모델을 테스트합니다.

* 훈련 셋, 테스트 셋 총 2개만 사용하면 어떤가?
    * 훨씬 간단한 것 같지만, 이렇게 하지 않는 이유는 **모델을 개발할 때 항상 모델의 설정을 튜닝**하기 때문입니다. (Hyperparameter ; 층의 수나 유닛 층의 수
    * 검증 세트에서 모델의 성능을 평가하여 이런 튜닝을 수행합니다. 본질적으로 이런 튜닝도 어떤 파라미터 공간에서 좋은 설정을 찾는 **학습** 입니다.
    * 검증 세트의 성능을 기반으로 모델의 설정을 튜닝하면 검증 세트로 모델을 직접 훈련하지 않더라도 빠르게 검증 세트에 과대적합될 수 있습니다.
    

* 정보 누설 (Information Leak) : 검증 세트의 모델 성능에 기반하여 모델의 하이퍼파라미터를 조정할 때마다 검증 데이터에 관한 정보가 모델로 새는 것입니다. 검증 데이터 세트에 맞춰 최적화했기 때문에 검증 데이터에 의도적으로 잘 수행되는 모델이 만들어집니다. 검증데이터가 아니고 완전히 새로운 데이터에 대한 성능이 관심 대상이라면 모델을 평가하기 위해 이전에 본 적 없는 완전히 다른 데이터셋을 사용해야합니다. 이것이 바로 *테스트 데이터*입니다.


* 데이터를 훈련, 검증, 테스트 세트로 나누는 것은 간단해보일 수 있지만 몇가지 고급 기법을 사용하면 도움이 됩니다. : hold-out validation, K-fold cross-validation, Shuffling

#### 단순 홀드아웃 검증 (Hold-out Variation)
데이터의 일정량을 테스트 세트로 떄어놓는다. 정보 누설을 막기 위해 테스트 세트를 사용하여 모델 튜닝을 하면 안됩니다. 이런 이유로 검증 세트도 따로 떼어 놓아야 합니다.

In [ ]:
import numpy as np

num_validation_samples = 10000

np.random.shuffle(data)

# 검증 세트를 만듭니다.
validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

# 훈련 세트를 만듭니다.
training_data = data[:]

# 훈련 세트에서 모델을 훈련하고 검증세트로 평가합니다.
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

### 여기서 모델을 튜닝하고
### 다시 훈련 -> 평가 -> 튜닝 의 반복

# 하이퍼파라미터 튜닝이 끝나면 테스트 데이터를 제외한 모든 데이터를 사용하여 모델을 다시 훈련시킵니다.
model = get_model()
model.train(np.concatenate([training_data,
                           validation_data]))
test_score = model.evaluate(testdata)

이 평가 방법은 단순해서 한가지 단점이 있습니다. 데이터가 적을 때는 검증 세트와 테스트의 샘플이 너무 적어 주어진 전체 데이터를 통계적으로 대표하지 못할 수도 있습니다.

#### K-겹 교차 검증 (K-Fold Cross-validation)


데이터를 동일한 크기를 가진 K개 분할로 나눕니다. 각 분할 i 에 대하여 남은 K - 1개의 분할로 모델을 훈련하고 분할 i에서 모델을 평가합니다. 최종 점수는 **이렇게 얻은 K개의 점수를 평균합니다.** 모델의 성능이 데이터 분할에 따라 편차가 클 때 도움이 됩니다. 단순 홀드아웃 검증과 똑같이 모델의 튜닝에 별개의 검증세트를 사용하게 됩니다.

In [ ]:
k = 4
num_validation_samples =len(data) / 4

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold:num_validation_samples * (fold + 1)]
    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold+1):]
    
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)
validation_scores.append(validation_score)

validation_score  =  np.average(validation_score)

model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

#### **셔플링**을 사용한 반복 K-겹 교차 검증

이 방법은 비교적 *가용 데이터가 적고 가능한 정환한 모델을 평가하고자 할 때* 사용합니다. **캐글 경연에서는 이 방법이 아주 크게 도움이 됩니다.** 
이 방법은 K-겹 교차 검증을 여러번 적용하게 되 K개의 분할로 나누기 전에 매번 데이터를 무작위로 섞습니다. 최종 점수는 모든 K-겹 교차 검증을 실행해서 얻은 점수의 평균이 됩니다. P X K 의 모델로 훈련하고 평가하므로 비용이 많이듭니다. (P : 반복 횟수)

### 기억해야할 것
평가 방식을 선택할 때 다음 사항을 유념해야 합니다.
1. **대표성 있는 데이터**
2. **시간의 방향** : 과거로부터 미래를 예측하려고 한다면, 데이터를 분할하기 전에 무작위로 섞어서는 저랟로 안됩니다. 이렇게 하면 미래의 정보가 누설되기 떄문입니다. 즉, 모델이 사실상 미래 데이터에서 훈련될 것입니다. 이런 문제에서는 훈련 세트에 있는 데이터보다 테스트 세트에 있는 모든 데이터가 미래의 것이어야 합니다.
3. **데이터 중복**

## 4.3 데이터 전처리, 특성 공학, 특성 학습

*많은 데이터 전처리와 특성 공학기법은 **특정 도메인에 종속적**입니다.*

### 신경망을 위한 데이터 전처리

데이터 전처리의 목적은 주어진 원본 데이터를 신경망에 적용하기 쉽도록 만드는 것입니다.

#### 벡터화 (Vectorization)
신경망에서 모든 입력과 타깃은 부동 소수 데이터로 이루어진 텐서어야 합니다. (특정 경우 정수로 이루어진 텐서) 처리해야할 것이 무엇이든지 먼저 텐서로 변환해야 합니다.

#### 값 정규화 (Normalization)
일반적으로 비교적 큰 값이나 균일하지 않은 데이터를 신경망에 주입하는 것은 위험합니다. 이렇게 하면 업데이트할 그레디언트가 커져 네트워크가 수렴하는 것을 방해합니다.
* 작은 값을 취합니다. 일반적으로 값이 *0 ~ 1* 사이어야 합니다.
* 균일해야합니다. 즉 모든 특성이 대체로 비슷한 범위를 가져야 합니다

추가적으로 다음으로 나오는 엄격한 정규화 방법은 꼭 필수적이지는 않지만 자주 사용되고 도움이 될 수 있습니다.
* 각 특성별로 평균이 0이 되도록 정규합니다.
* 각 특성별로 표준 편차가 1이 되도록 정규화합니다.

In [ ]:
x -= x.mean(axis = 0)
x /= x.std(axis = 0)

#### 누락된 값 다루기

어떤 특성이 모든 샘플에는 들어있지 않는 경우가 있을 수도 있습니다. 일반적으로 신경망에서 0이 사전 정의된 의미있는 값이 아니라면 누락된 값을 0으로 입력해도 괜찮습니다. 네트워크가 0이 누락된 데이터를 의미한다는 것을 학습하면 **이 값을 무시하기 시작할 것입니다.** 반면 네트워크가 누락된 값이 없는 데이터에서 훈련을 받았다면, 이 네트워크는 누락된 값을 무시하는 법을 알지 못합니다. 이런 경우, 누락된 값이 있는 훈련 샘플을 **고의적으로 만들어야합니다.** 훈련 샘플의 일부를 여러벌 복사해서 테스트 데이터에서 빠질 것 같은 특성을 제거합니다.

### 특성 공학

특성 공학은 데이터와 머신 러닝 알고리즘에 관